In [1]:
import subprocess
import zenhan
import os
import sys
import re
import unicodedata
import argparse
import pandas as pd
import pickle
import json
import tqdm

In [2]:
# テキストの取得
#企業リストを取得
#企業リストを取得
company_list = pd.read_csv("../data/new2/df_achieve/company_list.csv",index_col=0)
company_list=company_list.drop(14350)
company_list=company_list.drop(52080)
company_list=company_list.drop(95080)
with open('../data/text_data_ver2/non_company_list.pkl', 'rb') as f:
    non_list = pickle.load(f)
for com in non_list:
    company_list=company_list.drop(com)

In [22]:
#目次の確認用のdict
word_dic={}
non_list=[]
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)

    #企業の決算短信を一つ一つ見る
    for code in df['pdf_code']:
        json_path = '../data/text_data_ver2/text_and_bert_label_ver1/'+str(company)+'/'+str(code)+'.json'
        try:
            with open(json_path, 'r') as f:
                json_load = json.load(f)

            #まず目次の確認
            for text_dic in json_load["all_text"]:
            #text_labelにテキストがある目次だけ取得
                if ('財務諸表' in text_dic["contents"])or('会計基準' in text_dic["contents"]):
                    break
                else:
                    if text_dic["text_label"]!=[]:
                        if text_dic["contents"] in word_dic.keys():
                            word_dic[text_dic["contents"]] +=1
                        else:
                            word_dic[text_dic["contents"]] =1
        except:
            non_list.append(str(company)+"_"+str(code))

100%|██████████████████████████████████████| 2355/2355 [00:04<00:00, 512.44it/s]


In [24]:
#辞書をソートする
dic2 = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)

In [25]:
dic2

[('目次より前', 42103),
 ('（１）経営成績に関する説明', 28437),
 ('（２）財政状態に関する説明', 26949),
 ('（３）連結業績予想などの将来予測情報に関する説明', 26626),
 ('１．当四半期決算に関する定性的情報', 17824),
 ('（１）当期の経営成績の概況', 7841),
 ('（２）当期の財政状態の概況', 6902),
 ('（４）今後の見通し', 6391),
 ('（３）当期のキャッシュ・フローの概況', 5855),
 ('（５）利益配分に関する基本方針及び当期・次期の配当', 1681),
 ('（３）当期のキャッシュ', 1317),
 ('１．経営成績等の概況', 1291),
 ('\n１．経営成績等の概況', 1276),
 ('２．企業集団の状況', 1067),
 ('（２）連結業績予想などの将来予測情報に関する説明', 640),
 ('（３）利益配分に関する基本方針及び当期・次期の配当', 615),
 ('（１）経営成績に関する分析', 601),
 ('(3)連結業績予想などの将来予測情報に関する説明', 509),
 ('（２）財政状態に関する分析', 503),
 ('(2)財政状態に関する説明', 464),
 ('（１）会社の経営の基本方針', 460),
 ('1.経営成績等の概況', 449),
 ('（５）利益配分に関する基本方針及び当期', 419),
 ('（４）連結業績予想などの将来予測情報に関する説明', 405),
 ('(1)経営成績に関する説明', 358),
 ('（４）事業等のリスク', 316),
 ('（３）利益配分に関する基本方針及び当期', 311),
 ('（１）連結経営成績に関する説明', 297),
 ('（３）キャッシュ', 297),
 ('（重要な後発事象）', 291),
 ('（２）連結財政状態に関する説明', 285),
 ('（２）目標とする経営指標', 271),
 ('（５）継続企業の前提に関する重要事象等', 270),
 ('（１）連結経営成績に関する定性的情報', 258),
 ('1.当四半期決算に関する定性的情報', 231),
 ('（追加情報）', 231),
 ('\n１．当四半期決算に関する定

In [26]:
#まず目次の確認
for text_dic in json_load["all_text"]:
    print(text_dic["contents"])

目次より前
１．経営成績等の概況
（１）当期の経営成績の概況
（２）当期の財政状態の概況
（３）当期のキャッシュ
（４）今後の見通し
２．会計基準の選択に関する基本的な考え方
（重要な後発事象）


In [7]:
#labelの確認

In [28]:
df

,uriage,pred,achieve,f_score,j_score,word_cnt,pdf_code,title,tuki_pred,bert_pos_score,bert_neg_score,bert_score
2017-07-31,41153,160000.0,-0.742794,-0.111576,0.065551,717,20170731443984,平成30年3月期 第1四半期決算短信〔日本基準〕（連結）,160000.0,0.780488,0.365854,0.414634
2017-10-31,74863,160000.0,-0.532106,-0.113082,0.049889,902,20171030403832,平成30年3月期 第2四半期決算短信〔日本基準〕（連結）,160000.0,0.750000,0.403846,0.346154
2018-01-31,121997,160000.0,-0.237519,-0.100139,0.063978,719,20180131460096,平成30年3月期 第3四半期決算短信〔日本基準〕（連結）,160000.0,0.914286,0.228571,0.685714
2018-05-11,161673,180000.0,0.010456,-0.109687,0.047009,1404,20180511435494,平成30年3月期 決算短信〔日本基準〕（連結）,180000.0,0.731707,0.353659,0.378049
2018-07-31,41979,180000.0,-0.766783,-0.101562,0.042188,640,20180731488280,平成31年3月期 第1四半期決算短信〔日本基準〕（連結）,180000.0,0.736842,0.394737,0.342105
2018-10-31,79161,180000.0,-0.560217,-0.124231,0.034440,813,20181031426738,平成31年3月期 第2四半期決算短信〔日本基準〕（連結）,180000.0,0.640000,0.500000,0.140000
2019-01-31,133555,180000.0,-0.258028,-0.102857,0.031429,700,20190131467263,平成31年3月期 第3四半期決算短信〔日本基準〕（連結）,180000.0,0.631579,0.552632,0.078947
2019-05-13,177648,185000.0,-0.013067,-0.108452,0.035901,1337,20190513422733,2019年3月期 決算短信〔日本基準〕（連結）,185000.0,0.705128,0.461538,0.243590
2019-07-31,46155,185000.0,-0.750514,-0.093939,0.027273,660,20190731479653,2020年3月期 第1四半期決算短信〔日本基準〕（連結）,185000.0,0.594595,0.459459,0.135135
2019-10-31,86222,185000.0,-0.533935,-0.106977,0.020930,860,20191031417037,2020年3月期 第2四半期決算短信〔日本基準〕（連結）,185000.0,0.632653,0.510204,0.122449


In [35]:
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)

    #スコアを算出
    pos_list=[]
    neg_list=[]
    score_list=[]
    #企業の決算短信を一つ一つ見る
    for code in df['pdf_code']:
        json_path = '../data/text_data_ver2/text_and_bert_label_ver1/'+str(company)+'/'+str(code)+'.json'
        try:
            with open(json_path, 'r') as f:
                json_load = json.load(f)

            neg_cnt=0
            pos_cnt=0
            text_cnt=0
            for text_dic in json_load["all_text"]:
                #     print(text_dic["contents"])
                #text_labelにテキストがある目次だけ取得
                if ('財務諸表' in text_dic["contents"])or('会計基準' in text_dic["contents"]):
                    break
                else:
                    if text_dic["text_label"]!=[]:
                        for text in text_dic["text_label"]:
                            text_cnt+=1
                            if text["label"][0]==1:
                                neg_cnt+=1
                            if text["label"][1]==1:
                                pos_cnt+=1

            if text_cnt!=0:
                neg_list.append(neg_cnt/text_cnt)
                pos_list.append(pos_cnt/text_cnt)
                score_list.append((pos_cnt-neg_cnt)/text_cnt)
            else:
                neg_list.append(-1)
                pos_list.append(-1)
                score_list.append(-1)
        except:
            neg_list.append(-1)
            pos_list.append(-1)
            score_list.append(-1)
        
    df["bert_pos_score"]=pos_list
    df["bert_neg_score"]=neg_list
    df["bert_score"]=score_list
    #break
#     df.to_csv(df_path)
    

100%|██████████████████████████████████████| 2355/2355 [00:06<00:00, 388.81it/s]


In [36]:
df

,uriage,pred,achieve,f_score,j_score,word_cnt,pdf_code,title,tuki_pred,bert_pos_score,bert_neg_score,bert_score
2017-07-31,41153,160000.0,-0.742794,-0.111576,0.065551,717,20170731443984,平成30年3月期 第1四半期決算短信〔日本基準〕（連結）,160000.0,0.657143,0.285714,0.371429
2017-10-31,74863,160000.0,-0.532106,-0.113082,0.049889,902,20171030403832,平成30年3月期 第2四半期決算短信〔日本基準〕（連結）,160000.0,0.608696,0.304348,0.304348
2018-01-31,121997,160000.0,-0.237519,-0.100139,0.063978,719,20180131460096,平成30年3月期 第3四半期決算短信〔日本基準〕（連結）,160000.0,0.714286,0.114286,0.600000
2018-05-11,161673,180000.0,0.010456,-0.109687,0.047009,1404,20180511435494,平成30年3月期 決算短信〔日本基準〕（連結）,180000.0,0.591837,0.204082,0.387755
2018-07-31,41979,180000.0,-0.766783,-0.101562,0.042188,640,20180731488280,平成31年3月期 第1四半期決算短信〔日本基準〕（連結）,180000.0,0.588235,0.323529,0.264706
2018-10-31,79161,180000.0,-0.560217,-0.124231,0.034440,813,20181031426738,平成31年3月期 第2四半期決算短信〔日本基準〕（連結）,180000.0,0.500000,0.434783,0.065217
2019-01-31,133555,180000.0,-0.258028,-0.102857,0.031429,700,20190131467263,平成31年3月期 第3四半期決算短信〔日本基準〕（連結）,180000.0,0.529412,0.441176,0.088235
2019-05-13,177648,185000.0,-0.013067,-0.108452,0.035901,1337,20190513422733,2019年3月期 決算短信〔日本基準〕（連結）,185000.0,0.541667,0.395833,0.145833
2019-07-31,46155,185000.0,-0.750514,-0.093939,0.027273,660,20190731479653,2020年3月期 第1四半期決算短信〔日本基準〕（連結）,185000.0,0.424242,0.424242,0.000000
2019-10-31,86222,185000.0,-0.533935,-0.106977,0.020930,860,20191031417037,2020年3月期 第2四半期決算短信〔日本基準〕（連結）,185000.0,0.466667,0.444444,0.022222


## bert_score = (pos-neg)/text

In [28]:
#pos_negのスコアを算出
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)

    #スコアを算出
    score_list=[]

    #企業の決算短信を一つ一つ見る
    for code in df['pdf_code']:
        json_path = '../data/text_data_ver2/text_and_bert_label/'+str(company)+'/'+str(code)+'.json'
        with open(json_path, 'r') as f:
            json_load = json.load(f)
        
        neg_cnt=0
        pos_cnt=0
        text_cnt=0
        for text_dic in json_load["all_text"]:
        #     print(text_dic["contents"])
        
            if text_dic["text_label"]!=[]:
                for text in text_dic["text_label"]:
                    text_cnt+=1
                    if text["label"][0]==1:
                        neg_cnt+=1
                    if text["label"][1]==1:
                        pos_cnt+=1

        if text_cnt!=0:
            score_list.append((pos_cnt-neg_cnt)/text_cnt)
        else:
            score_list.append(-1)
    
    df["bert_score"]=score_list
    df.to_csv(df_path)

100%|██████████████████████████████████████| 2355/2355 [00:11<00:00, 197.17it/s]


In [27]:
df

,uriage,pred,achieve,f_score,j_score,word_cnt,pdf_code,title,tuki_pred,bert_pos_score,bert_neg_score,bert_score
2017-08-04,56844,250000.0,-0.772624,-0.013423,0.050336,596,20170619410034,平成30年3月期 第1四半期決算短信〔日本基準〕(連結),250000.0,0.648649,0.459459,0.189189
2017-11-06,120458,250000.0,-0.518168,0.009419,0.053375,637,20170905468428,平成30年3月期 第2四半期決算短信〔日本基準〕(連結),250000.0,0.648649,0.432432,0.216216
2018-02-09,198323,250000.0,-0.206708,-0.011905,0.050595,672,20171215436591,平成30年3月期 第3四半期決算短信〔日本基準〕(連結),250000.0,0.704545,0.409091,0.295455
2018-05-10,254783,267000.0,0.019132,-0.062235,0.043140,1414,20180402403415,平成30年3月期 決算短信〔日本基準〕(連結),267000.0,0.726190,0.357143,0.369048
2018-08-03,57957,267000.0,-0.782933,-0.045007,0.040788,711,20180627471686,平成31年3月期 第1四半期決算短信〔日本基準〕(連結),267000.0,0.681818,0.477273,0.204545
2018-11-05,119202,267000.0,-0.553551,-0.048193,0.038822,747,20180921409376,平成31年3月期 第2四半期決算短信〔日本基準〕(連結),267000.0,0.541667,0.520833,0.020833
2019-02-08,197783,267000.0,-0.259240,-0.056235,0.050122,818,20190125463477,平成31年3月期 第3四半期決算短信〔日本基準〕(連結),267000.0,0.607843,0.529412,0.078431
2019-05-13,256151,286000.0,-0.040633,-0.066961,0.047378,1583,20190510420409,2019年3月期 決算短信〔日本基準〕(連結),286000.0,0.709677,0.408602,0.301075
2019-08-02,62644,286000.0,-0.780965,-0.062252,0.049007,755,20190723474600,2020年3月期 第1四半期決算短信〔日本基準〕(連結),286000.0,0.617021,0.510638,0.106383
2019-11-05,126884,286000.0,-0.556350,-0.030750,0.051661,813,20191021410184,2020年3月期 第2四半期決算短信〔日本基準〕(連結),286000.0,0.666667,0.509804,0.156863
